<a href="https://colab.research.google.com/github/hoangvn111/Project-Analyzing-CIA-Factbook-Data-Using-SQL/blob/master/Project_Analyzing_CIA_Factbook_Data_Using_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import sqlite3

conn = sqlite3.connect('/content/drive/MyDrive/my_datasets/Project_ Analyzing CIA Factbook Data Using SQL/factbook.db')
print("Opened database successfully")

Opened database successfully


# Introduction

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:

* **population** — the global population.
* **population_growth** — the annual * * ?population growth rate, as a percentage.
* **area** — the total land and water area.

In [3]:
import pandas as pd
# Explore the database
df = pd.read_sql_query('''SELECT * 
                          FROM   facts
                          LIMIT  5;''', conn)
df

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
1,2,al,Albania,28748,27398,1350,3029278,0.30,12.92,6.58,3.30
2,3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
3,4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.00
4,5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


# Summary Statistics

Executing our previous query, we see the first few rows of our facts table look like this:

Here are the descriptions for some of the columns:

* **name** — the name of the country.
* **area—** the country's total area (both land and water).
* **area_land** — the country's land area in square kilometers.
* **area_water** — the country's waterarea in square kilometers.
* **population** — the country's population.
* **population_growth** — the country's population growth as a percentage.
* **birth_rate** — the country's birth rate, or the number of births per year per 1,000 people.
* death_rate — the country's death rate, or the number of death per year per 1,000 people.

In [4]:
# Write a single query that returns maximum polution
max_population = pd.read_sql_query('''SELECT MAX(population) 
                                        FROM facts;''', conn)
max_population

,MAX(population)
0,7256490011


In [5]:
# Write a single query that returns minimum polution
min_population = pd.read_sql_query('''SELECT MIN(population) 
                                        FROM facts;''', conn)
min_population

,MIN(population)
0,0


In [6]:
# Write a single query that returns maximum polution growth
max_population_growth = pd.read_sql_query('''SELECT MAX(population_growth) 
                                               FROM facts;''', conn)
max_population_growth

,MAX(population_growth)
0,4.02


In [7]:
# # Write a single query that returns minimum polution growth
min_population_growth = pd.read_sql_query('''SELECT MIN(population_growth) 
                                               FROM facts;''', conn)
min_population_growth

,MIN(population_growth)
0,0.0


# Exploring Outliers

We see a few interesting things in the summary statistics on the previous screen:

* There's a country with a population of 0
* There's a country with a population of 7256490011 (or more than 7.2 billion people)

In [8]:
# Write a query that returns the countries with the maximum population.
max_population = pd.read_sql_query('''SELECT *, MAX(population) 
                                        FROM facts;''', conn)
max_population

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,MAX(population)
0,261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None,7256490011


In [9]:
# Write a query that returns the countries with the minimum population.
min_population = pd.read_sql_query('''SELECT *, MIN(population) 
                                        FROM facts;''', conn)
min_population

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,MIN(population)
0,250,ay,Antarctica,None,280000,None,0,None,None,None,None,0


In [10]:
# Write a single query that returns maximum polution excluding the row for the whole world
max_population = pd.read_sql_query('''SELECT *, MAX(population) 
                                        FROM facts
                                       WHERE name <> 'World';''', conn)
max_population

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,MAX(population)
0,37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44,1367485388


In [11]:
# Write a single query that returns minimum polution excluding the row for the whole world
min_population = pd.read_sql_query('''SELECT *, MIN(population) 
                                        FROM facts
                                       WHERE name <> 'World';''', conn)
min_population

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,MIN(population)
0,250,ay,Antarctica,None,280000,None,0,None,None,None,None,0


In [12]:
# Calculate the average value for the following columns:
avg_population = pd.read_sql_query(''' SELECT AVG(population)
                                         FROM facts
                                        WHERE name <> 'World';''', conn)
avg_population

,AVG(population)
0,3.224267e+07


In [13]:
# Calculate the average value for the following columns:
avg_area = pd.read_sql_query(''' SELECT AVG(area)
                                         FROM facts
                                        WHERE name <> 'World';''', conn)
avg_area

,AVG(area)
0,555093.546185


# Finding Densely Populated Countries


We'll identify countries that have the following:

* Above-average values for population.
* Below-average values for area.

In [24]:
# Write a single query return countries have population > avg_population
above_avg_population = pd.read_sql_query(''' SELECT *
                                                FROM facts
                                               WHERE name <> 'World' 
                                                 AND population > (SELECT AVG(population)
                                                                     FROM facts);''', conn)
above_avg_population

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,14,bg,Bangladesh,148460,130170.0,18290.0,168957745,1.60,21.14,5.61,0.46
1,24,br,Brazil,8515770,8358140.0,157630.0,204259812,0.77,14.46,6.58,0.14
2,37,ch,China,9596960,9326410.0,270550.0,1367485388,0.45,12.49,7.53,0.44
3,40,cg,"Congo, Democratic Republic of the",2344858,2267048.0,77810.0,79375136,2.45,34.88,10.07,0.27
4,53,eg,Egypt,1001450,995450.0,6000.0,88487396,1.79,22.90,4.77,0.19
5,58,et,Ethiopia,1104300,NaN,104300.0,99465819,2.89,37.27,8.19,0.22
6,61,fr,France,643801,640427.0,3374.0,66553766,0.43,12.38,9.16,1.09
7,65,gm,Germany,357022,348672.0,8350.0,80854408,0.17,8.47,11.42,1.24
8,77,in,India,3287263,2973193.0,314070.0,1251695584,1.22,19.55,7.32,0.04
9,78,id,Indonesia,1904569,1811569.0,93000.0,255993674,0.92,16.72,6.37,1.16


In [23]:
# Write a single query return countries have population < avg_population
below_avg_population = pd.read_sql_query(''' SELECT *
                                                FROM facts
                                               WHERE name <> 'World' 
                                                 AND population < (SELECT AVG(population)
                                                                     FROM facts);''', conn)
below_avg_population

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,1,af,Afghanistan,652230.0,652230.0,0.0,32564342,2.32,38.57,13.89,1.51
1,2,al,Albania,28748.0,27398.0,1350.0,3029278,0.30,12.92,6.58,3.30
2,3,ag,Algeria,2381741.0,2381741.0,0.0,39542166,1.84,23.67,4.31,0.92
3,4,an,Andorra,468.0,468.0,0.0,85580,0.12,8.13,6.96,0.00
4,5,ao,Angola,1246700.0,1246700.0,0.0,19625353,2.78,38.78,11.49,0.46
...,...,...,...,...,...,...,...,...,...,...,...
213,247,vq,Virgin Islands,1910.0,346.0,1564.0,103574,0.59,10.31,8.54,7.67
214,250,ay,Antarctica,NaN,280000.0,NaN,0,NaN,NaN,NaN,NaN
215,251,gz,Gaza Strip,360.0,360.0,0.0,1869055,2.81,31.11,3.04,0.00
216,254,we,West Bank,5860.0,5640.0,220.0,2785366,1.95,22.99,3.50,0.00


In [22]:
# Write a single query return countries have population > avg_area
above_avg_area = pd.read_sql_query(''' SELECT *
                                         FROM facts
                                        WHERE name <> 'World' 
                                          AND population > (SELECT AVG(area)
                                                                     FROM facts);''', conn)
above_avg_area

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,1,af,Afghanistan,652230.0,652230.0,0.0,32564342,2.32,38.57,13.89,1.51
1,2,al,Albania,28748.0,27398.0,1350.0,3029278,0.30,12.92,6.58,3.30
2,3,ag,Algeria,2381741.0,2381741.0,0.0,39542166,1.84,23.67,4.31,0.92
3,5,ao,Angola,1246700.0,1246700.0,0.0,19625353,2.78,38.78,11.49,0.46
4,7,ar,Argentina,2780400.0,2736690.0,43710.0,43431886,0.93,16.64,7.33,0.00
...,...,...,...,...,...,...,...,...,...,...,...
168,205,mc,Macau,28.0,28.0,0.0,592731,0.80,8.88,4.22,3.37
169,246,rq,Puerto Rico,13791.0,8870.0,4921.0,3598357,0.60,10.86,8.67,8.15
170,251,gz,Gaza Strip,360.0,360.0,0.0,1869055,2.81,31.11,3.04,0.00
171,254,we,West Bank,5860.0,5640.0,220.0,2785366,1.95,22.99,3.50,0.00


In [21]:
# Write a single query return countries have population < avg_area
below_avg_area = pd.read_sql_query(''' SELECT *
                                         FROM facts
                                        WHERE name <> 'World' 
                                          AND population < (SELECT AVG(area)
                                                                     FROM facts);''', conn)
below_avg_area

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,4,an,Andorra,468.0,468.0,0.0,85580,0.12,8.13,6.96,0.00
1,6,ac,Antigua and Barbuda,442.0,442.0,0.0,92436,1.24,15.85,5.69,2.21
2,12,bf,"Bahamas, The",13880.0,10010.0,3870.0,324597,0.85,15.50,7.05,0.00
3,15,bb,Barbados,430.0,430.0,0.0,290604,0.31,11.87,8.44,0.30
4,18,bh,Belize,22966.0,22806.0,160.0,347369,1.87,24.68,5.97,0.00
...,...,...,...,...,...,...,...,...,...,...,...
63,242,aq,American Samoa,199.0,199.0,0.0,54343,0.30,22.89,4.75,21.13
64,243,gq,Guam,544.0,544.0,0.0,161785,0.54,16.82,5.12,6.34
65,245,cq,Northern Mariana Islands,464.0,464.0,0.0,52344,2.18,18.32,3.71,7.16
66,247,vq,Virgin Islands,1910.0,346.0,1564.0,103574,0.59,10.31,8.54,7.67
